In [5]:
import pandas as pd
from tqdm import tqdm
import numpy as np

1. Phase별 데이터 전처리 + write

In [ ]:
base_path = '../data/open_track1/'
df = pd.read_csv(base_path + "train.csv")
df = df.sort_values(["game_episode", "time_seconds"]).reset_index(drop=True)

ph = 0
past_game_id = 0
past_team_id = 0
phase = []

for row in df.itertuples():
    if past_team_id != row.team_id:
        ph += 1
    
    if row.Index == 0:
        phase.append(str(row.game_id) + '_' + str(ph))
        continue
    if past_game_id != row.game_id:
        ph = 1

    

    phase.append(str(row.game_id) + '_' + str(ph))

    past_game_id = row.game_id
    past_team_id = row.team_id


df['phase'] = phase
# print(df.head())

# df.to_csv('phase_train.csv', index=False)


   game_id  period_id  episode_id  time_seconds  team_id  player_id  \
0   126283          1           1         0.667     2354     344559   
1   126283          1           1         3.667     2354     250036   
2   126283          1           1         4.968     2354     500145   
3   126283          1           1         8.200     2354     500145   
4   126283          1           1        11.633     2354     142106   

   action_id type_name result_name    start_x    start_y      end_x  \
0          0      Pass  Successful  52.418205  33.485444  31.322445   
1          2      Pass  Successful  32.013240  38.100808  37.371285   
2          4     Carry         NaN  37.371285  30.632980  38.391570   
3          5      Pass  Successful  38.391570  24.613144  34.573350   
4          7      Pass  Successful  34.578705   6.058256  21.274470   

       end_y  is_home game_episode     phase  
0  38.274752     True     126283_1  126283_1  
1  30.632980     True     126283_1  126283_1  
2  24